## 线性回归的简洁实现

- 通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集
- 使用pytorch的nn module 及 数据预处理模块

In [1]:
# 导入工具包
import numpy as np
import torch
from torch.utils import data  # 数据处理包
from d2l import torch as d2l

In [5]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 调用框架中现有的API来读取数据

In [6]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 每次随机挑选batchsize个样本

batch_size = 10
data_iter = load_array((features, labels), batch_size)  # (features, labels) 作为一个list

"""
next() 返回迭代器的下一个项目
next() 函数要和生成迭代器的iter() 函数一起使用

list、tuple等都是可迭代对象，通过iter()函数获取这些可迭代对象的迭代器
然后，对获取到的迭代器不断使⽤next()函数来获取下⼀条数据
iter()函数实际上就是调⽤了可迭代对象的 iter ⽅法
"""
next(iter(data_iter))

[tensor([[-0.5482, -2.2417],
         [-0.1411,  1.8357],
         [ 1.1583,  0.4196],
         [ 0.2179,  0.3887],
         [-0.0920,  1.4153],
         [ 0.2690, -1.5231],
         [ 0.5779, -0.9576],
         [ 0.5284, -0.8330],
         [-0.5534,  0.2808],
         [-1.4085, -1.3706]]),
 tensor([[10.7314],
         [-2.3108],
         [ 5.0895],
         [ 3.3191],
         [-0.7862],
         [ 9.9124],
         [ 8.6211],
         [ 8.0891],
         [ 2.1417],
         [ 6.0319]])]

### 定义模型

- 对于标准深度学习模型, **使用框架的预定义好的层**
- 这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节

In [8]:
# nn是神经网络的缩写
from torch import nn

"""
Sequential类将多个层串联在一起 list of layers
当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推
在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential
但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。
"""
net = nn.Sequential(nn.Linear(2, 1)) # 输入为2，输出为1

### 初始化模型参数

In [9]:
net[0].weight.data.normal_(0, 0.01)  # net[0] 对应第一层
net[0].bias.data.fill_(0)            # 使用替换方法normal_和fill_来重写参数值

tensor([0.])

### 定义损失函数

- 计算均方误差使用的是MSELoss类，也称为平方𝐿2范数

In [10]:
loss = nn.MSELoss()

### 实例化 SGD 实例

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # net.parameters所有参数  0.03学习率

### 训练

In [16]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000098
epoch 2, loss 0.000097
epoch 3, loss 0.000097


### 比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [17]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003,  0.0005])
b的估计误差： tensor([-0.0007])
